**Group 16**

Adham Mohmed Elsaied Elwakel 222100195,
Samaa Khaled Eltaky 222100761,
Habiba Ahmed Abdelnapy 222100471, 
Youssef Hussieny 222101943

### Part 1: PCA method with mean-filling 

In [6]:
import pandas as pd
ratings = pd.read_csv('../data/ratings.csv')

In [7]:
ratings

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858
...,...,...,...,...
32000199,200948,79702,4.5,1294412589
32000200,200948,79796,1.0,1287216292
32000201,200948,80350,0.5,1294412671
32000202,200948,80463,3.5,1350423800


#### 1- calculate the average for each of the target items (I1, and I2)

In [8]:
target_items = [1, 2]

In [9]:
item_means = ratings[ratings['movieId'].isin(target_items)].groupby('movieId')['rating'].mean()
item_means

movieId
1    3.897438
2    3.275758
Name: rating, dtype: float64

In [10]:
for item_id in target_items:
    if item_id in item_means.index:
        print(f"Mean rating for (I{target_items.index(item_id)+1}): {item_means[item_id]:.4f}")
    else:
        print(f"Item {item_id} not found in dataset.")

Mean rating for (I1): 3.8974
Mean rating for (I2): 3.2758


In [11]:
all_item_means = ratings.groupby('movieId')['rating'].mean()
print(f"\nTotal items with ratings: {len(all_item_means)}")


Total items with ratings: 84432


#### 2. Use the mean-filling method to replace the unspecified ratings of each of the target items (I1 and I2) with its corresponding mean value.

In [ ]:
# i will let this error just to let you know why we filtered our data later
user_item_matrix = ratings.pivot(index='userId', columns='movieId', values='rating')
user_item_matrix

In [12]:
#Select Top 1,000 Popular Movies (plus target items if they aren't in top 1000)
desired_item_count = 1000

In [13]:
item_counts = ratings['movieId'].value_counts()
top_movies = item_counts.nlargest(desired_item_count).index.tolist()

In [14]:
for t in target_items:
    if t not in top_movies:
        top_movies.append(t)

ratings_filtered = ratings[ratings['movieId'].isin(top_movies)]

In [15]:
#Select Top 100,000 Active Users
desired_user_count = 100000
user_counts = ratings_filtered['userId'].value_counts()
top_users = user_counts.nlargest(desired_user_count).index.tolist()

In [16]:
ratings_final = ratings_filtered[ratings_filtered['userId'].isin(top_users)]

In [17]:
ratings_final.shape

(15899442, 4)

In [18]:
ratings_final['userId'].nunique()

100000

In [19]:
ratings_final['movieId'].nunique()

1000

In [20]:
user_item_matrix = ratings_final.pivot(index='userId', columns='movieId', values='rating')
user_item_matrix

movieId,1,2,3,5,6,7,10,11,16,17,...,168252,171763,174055,176371,177765,187593,195159,202439,204698,207313
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,3.5,NaN,NaN,NaN,NaN,4.0,4.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,2.5,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,4.0,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,5.0,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200943,3.5,2.5,NaN,NaN,3.5,4.5,3.5,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200944,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
for item_id in target_items:
    if item_id in user_item_matrix.columns:
        # Recalculate mean on the filtered data to be accurate
        mean_val = user_item_matrix[item_id].mean() 
        
        user_item_matrix[item_id] = user_item_matrix[item_id].fillna(mean_val)
        print(f"Item {item_id}: Filled missing values with mean {mean_val:.4f}")
    else:
        print(f"Warning: Item {item_id} was lost during filtering (should not happen).")

Item 1: Filled missing values with mean 3.8833
Item 2: Filled missing values with mean 3.2343


In [22]:
user_item_matrix[target_items].isna().sum()

movieId
1    0
2    0
dtype: int64

#### 3- Calculate the average rating for each item 

In [23]:
all_item_means = user_item_matrix.mean(axis=0)

In [24]:
all_item_means.head()

movieId
1    3.883266
2    3.234285
3    3.045785
5    2.968495
6    3.882499
dtype: float64

In [25]:
all_item_means.shape

(1000,)

#### 4- For each item, calculate the difference between ratings and the mean rating of the item

In [26]:
centered_matrix = user_item_matrix - all_item_means
centered_matrix = centered_matrix.fillna(0)

In [27]:
centered_matrix

movieId,1,2,3,5,6,7,10,11,16,17,...,168252,171763,174055,176371,177765,187593,195159,202439,204698,207313
userId,,,,,,,,,,,,,,,,,,,,,
1,-4.440892e-16,4.440892e-16,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.095771,...,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,-4.440892e-16,2.657150e-01,0.0,0.0,0.000000,0.000000,0.593892,0.382083,0.0,1.095771,...,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
10,-1.383266e+00,-1.234285e+00,0.0,0.0,0.000000,0.000000,0.593892,0.000000,0.0,0.000000,...,0.11701,0.0,0.000000,-0.446614,0.0,0.0,0.0,0.0,0.0,0.0
13,-4.440892e-16,4.440892e-16,0.0,0.0,1.117501,0.000000,0.000000,-0.617917,0.0,0.000000,...,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
15,-4.440892e-16,4.440892e-16,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.595771,...,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200943,-3.832660e-01,-7.342850e-01,0.0,0.0,-0.382499,1.190252,0.093892,0.000000,0.0,1.095771,...,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
200944,1.167340e-01,4.440892e-16,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
200945,-4.440892e-16,4.440892e-16,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.00000,0.0,-3.278503,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


#### 5- compute the covariance for each two items

$$\text{Cov}(i, j) = \frac{\sum (\text{centered rating}_i \times \text{centered rating}_j)}{n - 1}$$

In [28]:
covariance_matrix = centered_matrix.cov()

In [29]:
covariance_matrix

movieId,1,2,3,5,6,7,10,11,16,17,...,168252,171763,174055,176371,177765,187593,195159,202439,204698,207313
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.418432,0.049546,0.011809,0.013562,0.015632,0.011164,0.031787,0.017110,0.009283,0.016394,...,0.012028,0.008232,0.006513,0.006794,0.014633,0.006937,0.011358,0.006268,0.005624,0.008137
2,0.049546,0.224879,0.015021,0.018806,0.010162,0.013708,0.029945,0.016461,0.009126,0.009451,...,0.006583,0.004138,0.002838,0.002844,0.005193,0.005362,0.003593,0.001706,0.003852,0.003776
3,0.011809,0.015021,0.081136,0.017217,0.005052,0.011113,0.010915,0.010921,0.004252,0.004265,...,0.001180,0.001032,0.000355,0.000608,0.000823,0.000689,0.000477,0.000458,0.000711,0.000571
5,0.013562,0.018806,0.017217,0.083338,0.005005,0.013676,0.011848,0.013854,0.003840,0.007465,...,0.001084,0.001017,0.000706,0.000905,0.000976,0.000160,0.000374,-0.000040,0.000344,0.000331
6,0.015632,0.010162,0.005052,0.005005,0.170923,0.005642,0.018178,0.007267,0.031026,0.005920,...,0.004100,0.003987,0.004366,0.005297,0.002390,0.001740,0.002080,0.003031,0.003244,0.002747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187593,0.006937,0.005362,0.000689,0.000160,0.001740,-0.000095,0.002020,0.000652,0.001731,0.000946,...,0.016645,0.010100,0.006315,0.006896,0.008128,0.073330,0.012060,0.005265,0.010212,0.007953
195159,0.011358,0.003593,0.000477,0.000374,0.002080,0.000286,0.001933,0.000608,0.001360,0.001387,...,0.013825,0.009090,0.006142,0.007699,0.010598,0.012060,0.067780,0.007930,0.007429,0.009245
202439,0.006268,0.001706,0.000458,-0.000040,0.003031,0.000145,0.001045,0.000666,0.002397,0.000898,...,0.006278,0.006780,0.006865,0.008117,0.005512,0.005265,0.007930,0.055058,0.010111,0.008723


In [30]:
covariance_matrix.iloc[:5, :5]

movieId,1,2,3,5,6
movieId,,,,,
1,0.418432,0.049546,0.011809,0.013562,0.015632
2,0.049546,0.224879,0.015021,0.018806,0.010162
3,0.011809,0.015021,0.081136,0.017217,0.005052
5,0.013562,0.018806,0.017217,0.083338,0.005005
6,0.015632,0.010162,0.005052,0.005005,0.170923


In [31]:
covariance_matrix.shape

(1000, 1000)

In [32]:
item_A = covariance_matrix.columns[0]
item_B = covariance_matrix.columns[1]

covariance_matrix.loc[item_A, item_B]

np.float64(0.04954585704488228)

#### 6- Generate the covariance matrix

In [33]:
cov_matrix = centered_matrix.cov()

In [34]:
nan_count = cov_matrix.isna().sum().sum()
nan_count

np.int64(0)

#### 7- Determine the top 5-peers and top 10-peers for each of the target items (i1 and I2) using the transformed representation (covariance matrix)

In [35]:
for item_id in target_items:
    if item_id in cov_matrix.index:
        item_covariances = cov_matrix[item_id]
        
        sorted_peers = item_covariances.sort_values(ascending=False)
        
        sorted_peers = sorted_peers.drop(item_id)
        
        top_5_peers = sorted_peers.head(5)
        top_10_peers = sorted_peers.head(10)
        for rank, (peer_id, score) in enumerate(top_5_peers.items(), 1):
            print(f"{rank}. Item {peer_id} (Covariance: {score:.4f})")

1. Item 3114 (Covariance: 0.1419)
2. Item 364 (Covariance: 0.1124)
3. Item 4886 (Covariance: 0.1122)
4. Item 480 (Covariance: 0.1092)
5. Item 6377 (Covariance: 0.1068)
1. Item 480 (Covariance: 0.0701)
2. Item 500 (Covariance: 0.0701)
3. Item 780 (Covariance: 0.0658)
4. Item 367 (Covariance: 0.0647)
5. Item 586 (Covariance: 0.0625)


#### 8- Determine reduced dimensional space for each user in case of using the top 5-peers

In [36]:
reduced_space_matrices = {}

for item_id in target_items:
    if item_id in cov_matrix.index:
        sorted_peers = cov_matrix[item_id].sort_values(ascending=False).drop(item_id)
        
        top_5_peer_ids = sorted_peers.head(5).index.tolist()
        
        reduced_matrix = centered_matrix[top_5_peer_ids]
        
        reduced_space_matrices[item_id] = reduced_matrix
        print(top_5_peer_ids)
        print(reduced_matrix.shape)

[3114, 364, 4886, 480, 6377]
(100000, 5)
[480, 500, 780, 367, 586]
(100000, 5)


#### 9- Use the results from point 8 compute the rating predictions od the original missing rating for each of the target items (I1 and I2) using the top 5-peers.

$$\text{Prediction}_{u, \text{target}} = \text{Mean}_{\text{target}} + \frac{\sum (\text{Covariance}_{\text{target, peer}} \times \text{CenteredRating}_{u, \text{peer}})}{\sum |\text{Covariance}_{\text{target, peer}}|}$$

In [37]:
for item_id in target_items:
    if item_id in reduced_space_matrices:
      
        reduced_matrix = reduced_space_matrices[item_id]
        
        peers_indices = reduced_matrix.columns
        weights = cov_matrix.loc[item_id, peers_indices]
        

        weighted_sum = reduced_matrix.dot(weights)
        
        sum_abs_weights = weights.abs().sum()
        
        target_mean = item_means[item_id]
        
        predicted_ratings = target_mean + (weighted_sum / sum_abs_weights)

        print(target_mean)
        print(sum_abs_weights)
        print(predicted_ratings.head(10))
        
    

3.8974375697494095
0.5826432315649777
userId
1     3.897438
3     4.025540
10    3.564518
13    3.897438
15    3.451000
16    3.130012
17    3.954804
18    4.477333
20    4.736929
25    3.805935
dtype: float64
3.2757576805978412
0.33315640816124126
userId
1     3.275758
3     3.280881
10    2.344455
13    3.275758
15    2.837956
16    3.275758
17    3.474500
18    3.275758
20    3.766197
25    3.234931
dtype: float64


#### 10- Determine reduced dimensional space for each user in case of using the top 10-peers.

In [38]:
reduced_spaces_10 = {}

for item_id in target_items:
    if item_id in cov_matrix.index:

        peers_indices = cov_matrix[item_id].nlargest(11).index.drop(item_id)
        

        reduced_matrix_10 = centered_matrix[peers_indices]
        
        reduced_spaces_10[item_id] = reduced_matrix_10
        

        print(peers_indices.tolist())
        print(reduced_matrix_10.shape)
        print(reduced_matrix_10.head())

[3114, 364, 4886, 480, 6377, 4306, 588, 1270, 260, 8961]
(100000, 10)
movieId  3114      364       4886     480       6377      4306      588   \
userId                                                                     
1         0.0  0.000000  0.000000  0.00000  0.000000  0.000000  0.000000   
3         0.0  0.185096  0.181879  0.30597  0.000000 -0.226372  0.294961   
10        0.0  0.185096  0.000000 -0.69403 -1.300599 -0.226372 -0.705039   
13        0.0  0.000000  0.000000  0.00000  0.000000  0.000000  0.000000   
15        0.0  0.000000 -2.318121  0.00000  0.000000 -1.726372  0.000000   

movieId      1270      260   8961  
userId                             
1        1.030749  0.893027   0.0  
3        0.030749 -0.106973   0.0  
10      -0.469251 -0.106973   0.0  
13      -0.969251  0.000000   0.0  
15      -0.469251  0.000000   0.0  
[480, 500, 780, 367, 586, 364, 736, 1580, 356, 588]
(100000, 10)
movieId     480       500       780       367       586       364       736   \


#### 11- Use the results from point 10 to compute the rating predictions of the original missing rating for each of target items (I1 and I2) using the top 10-peers.

In [39]:
for item_id in target_items:
    if item_id in reduced_spaces_10:

        reduced_matrix = reduced_spaces_10[item_id]
        
 
        peers_indices = reduced_matrix.columns
        weights = cov_matrix.loc[item_id, peers_indices]
       
        weighted_sum = reduced_matrix.dot(weights)
        
        sum_abs_weights = weights.abs().sum()
        
        target_mean = item_means[item_id]
        
        predicted_ratings = target_mean + (weighted_sum / sum_abs_weights)
        

        print(target_mean)
        print(sum_abs_weights)
        print(predicted_ratings.head(10))

3.8974375697494095
1.0619910672941997
userId
1     4.063227
3     3.967640
10    3.573925
13    3.812376
15    3.440984
16    3.476402
17    4.004242
18    4.393060
20    4.676654
25    3.754854
dtype: float64
3.2757576805978412
0.6027245685527863
userId
1     3.101778
3     3.319033
10    2.704104
13    3.360261
15    3.033763
16    3.053668
17    3.423663
18    3.495627
20    3.414970
25    3.253191
dtype: float64


#### 12- Compare the results of point 9 with results of point 11. comment your answer.

# in point 9: Due to the small k, 
# the calculations here is most strict in the similarity between the items, 
# which cause to have higher variance but less bias if the top 5 peers are very similar to the target item,
# and since the highest covariance between them was 0.1419, 
# it's obvious to have higher variance in the predictions
# but in point 10:adding more k, cause to have more smoothing in the predictions,
# which can decrease the variance in the output.